In [1]:
import formulas
import storage
import pandas as pd
import os
import re



if __name__ == '__main__':

    website_creator_path = os.getcwd()
    formula_file_path = website_creator_path + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_path + os.path.sep + 'sort_orders.csv'
    docs_dir = storage.get_docs_path(website_creator_path)
    
    df_formulas_by_year =  (
        formulas.get_formulas_by_year_df(formula_file_path))
    sort_orders = pd.read_csv(order_file_path)
    
    storage.delete_directory_if_it_exists(docs_dir)

    # Create formula by year summary
    formula_by_year_directories_df = (
        df_formulas_by_year[['State', 'Sub category 1', 'Sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    storage.create_sub_directories_from_df(
        docs_dir, formula_by_year_directories_df)
    regex_id_for_formula_dirs = '.*' + os.path.sep + 'Formulas*'
    storage.create_index_files(base_dir=docs_dir, 
                               folder_regex=regex_id_for_formula_dirs, 
                               book_collapse=True, 
                               df_sort_orders=sort_orders)
    # # # formulas.create_formula_files(docs_dir, df_formulas)


    # Create index files for State level menus
    regex_id_for_state_dirs =os.path.sep + '.{3}$'
    storage.create_index_files(base_dir=docs_dir, 
                               folder_regex=regex_id_for_state_dirs,
                               book_collapse=True, 
                               df_sort_orders=sort_orders)

    # df_calculus = formulas.df_calculus_summary(df_formulas_by_year)
    # styler_calculus = formulas.styler_calculus_summary(
    #     df_calculus, formulas.formulas_on_formula_sheet(df_formulas_by_year))
    # display(styler_calculus)

In [15]:


def create_formula_files(docs_dir, df_formulas):
    """Creates formula files in markdown format.  Files are created per state /
    subject  / categorycombination and stored in folders under docs_path
    according to this same combination"""

    formula_combinations = (df_formulas
                            [['State', 'Sub category 1', 'Sub category 2',
                              'Subject code', 'Category']].
                            drop_duplicates())
    
    for index, row in formula_combinations.iterrows():
        df = df_formulas[(
            (df_formulas['State'] == row['State']) &
            (df_formulas['Subject code'] == str(row['Subject code'])) & 
            (df_formulas['Category'] == row['Category']))]
        
        formula_2_col_is_empty = df['Formula_2'].dropna().empty    
        if formula_2_col_is_empty:
            df = df[['Formula_1']]
        else:
            df = df[['Formula_1', 'Formula_2']]
            
        formula_set_styler = (formulas.df_to_formula_styled_table(
            df=df, col_widths={'Formula_1':300, 'Formula_2':400},
            display_col_headers = False))
        output_string =  '#  \n<br>\n' + formula_set_styler.to_html()
        file_name = (docs_dir + os.path.sep +
                     row['State'] + os.path.sep + 
                     row['Sub category 1'] + os.path.sep +
                     row['Sub category 2'] + os.path.sep  + 
                     str(row['Subject code']) + os.path.sep  + 
                     row['Category']  + os.path.sep + 
                     row['Category']  + '.md')
        with open(file_name, "w") as text_file:
            text_file.write(output_string)


In [16]:
# df_formulas_by_yearb
create_formula_files(docs_dir, df_formulas_by_year)

In [13]:
len ('    """Creates formula files in markdown format.  Files are created per state /')

79